# Advanced Automatic Differentiation

## Overview

In this section, we review some advanced AD tricks especially their application on circuit simulations. With these advanced AD tricks, we can evaluate some quantum quantities more efficiently.

The advanced AD is possible in TensorCircuit, as we have implement several AD related API in backend agnostic way, the implementation of them closely follow the design philosophy of [jax AD implementation](https://jax.readthedocs.io/en/latest/notebooks/autodiff_cookbook.html).

## Setup

In [1]:
import numpy as np
import tensorcircuit as tc

K = tc.set_backend("tensorflow")

In [2]:
n = 6
nlayers = 3

Backend agnostic AD related APIs include the following

In [3]:
help(K.grad)
help(K.value_and_grad)
help(K.vectorized_value_and_grad)
help(K.vjp)
help(K.jvp)
help(K.jacfwd)
help(K.jacrev)
help(K.stop_gradient)
help(K.hessian)

Help on method grad in module tensorcircuit.backends.tensorflow_backend:

grad(f: Callable[..., Any], argnums: Union[int, Sequence[int]] = 0, has_aux: bool = False) -> Callable[..., Any] method of tensorcircuit.backends.tensorflow_backend.TensorFlowBackend instance
    Return the function which is the grad function of input ``f``.
    
    :Example:
    
    >>> f = lambda x,y: x**2+2*y
    >>> g = tc.backend.grad(f)
    >>> g(tc.num_to_tensor(1),tc.num_to_tensor(2))
    2
    >>> g = tc.backend.grad(f, argnums=(0,1))
    >>> g(tc.num_to_tensor(1),tc.num_to_tensor(2))
    [2, 2]
    
    :param f: the function to be differentiated
    :type f: Callable[..., Any]
    :param argnums: the position of args in ``f`` that are to be differentiated, defaults to be 0
    :type argnums: Union[int, Sequence[int]], optional
    :return: the grad function of ``f`` with the same set of arguments as ``f``
    :rtype: Callable[..., Any]

Help on method value_and_grad in module tensorcircuit.backends.t

## Forward AD

Using Jacobian vector product (``jvp``), we can compute the circuit gradient in the forward AD mode, which is more suitable when the number of output elements is much larger than the input.

Suppose we are going to evaluate $\partial \vert \psi(\theta) \rangle$, where $\psi(\theta) = U(\theta)\vert 0\rangle$ is the output state of some parameterized quantum circuit.

In [4]:
def ansatz(thetas):
    c = tc.Circuit(n)
    for j in range(nlayers):
        for i in range(n):
            c.rx(i, theta=thetas[j])
        for i in range(n - 1):
            c.cnot(i, i + 1)
    return c


def psi(thetas):
    c = ansatz(thetas)
    return c.state()

In [5]:
state, partial_psi_partial_theta0 = K.jvp(
    psi,
    K.implicit_randn([nlayers]),
    tc.array_to_tensor(np.array([1.0, 0, 0]), dtype="float32"),
)

We thus obtain $\frac{\partial \psi}{\partial \theta_0}$, since the tangent takes one in the first place and zero in other positions.

In [6]:
state.shape, partial_psi_partial_theta0.shape

(TensorShape([64]), TensorShape([64]))

## Jacobian

We can compute the Jacobian row by row or col by col using vmap together with reverse mode or forward mode AD.

We still use the above example, to calculate Jacobian $J_{ij}=\frac{\partial \psi_i}{\partial \theta_j}$.

In [7]:
thetas = K.implicit_randn([nlayers])

jac_fun = K.jit(K.jacfwd(psi))

jac_value = jac_fun(thetas)

In [8]:
%timeit jac_fun(thetas)

601 µs ± 36.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
jac_value.shape

TensorShape([64, 3])

We can also use reverse mode AD to obtain Jacobian.

In [10]:
jac_fun2 = K.jit(K.jacrev(psi))

jac_value2 = jac_fun2(thetas)

In [11]:
%timeit jac_fun2(thetas)

843 µs ± 9.95 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
jac_value2.shape

TensorShape([64, 3])

In [13]:
np.testing.assert_allclose(np.real(jac_value), jac_value2, atol=1e-5)

It is worth noting that forward mode AD Jacobian is faster since the result Jacobian is a tall matrix.

## Quantum Fisher Information

Quantum Fisher Information is a very important quantity in quantum computation, which can be utilized in so-called quantum natural gradient descent optimization as well as variational quantum dynamics. See [reference paper](https://arxiv.org/abs/1909.02108) for more details. 

There are several variants of QFI like object, and the core to evaluate is all related to $\langle \partial \psi \vert \partial \psi\rangle - \langle \partial \psi \vert \psi\rangle\langle \psi \vert \partial \psi\rangle$. Such quantity is easily obtained with advanced AD framework by first getting the Jacobian for the state and then vmap the inner product over Jacobian rows. The detailed implementation can be found at the codebase ``tensorcircuit/experimental.py``. We directly call the corresponding API in this note.

In [14]:
from tensorcircuit.experimental import qng

In [15]:
qfi_fun = K.jit(qng(psi))
qfi_value = qfi_fun(thetas)
qfi_value.shape

TensorShape([3, 3])

In [16]:
%timeit qfi_fun(thetas) # the speed is comparable with a simple Jacobian computation

609 µs ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Hessian

Hessian is defined as $\partial_{ij} \langle \psi(\theta)\vert H\vert \psi(\theta)\rangle$, where $ij$ is shorthand for $\theta_i\theta_j$.

In the following examples, we use $H=Z_0$ for simplicity.

In [17]:
def h(thetas):
    c = ansatz(thetas)
    return K.real(c.expectation_ps(z=[0]))


hess_f = K.jit(K.hessian(h))

In [18]:
hess_value = hess_f(thetas)
hess_value.shape

TensorShape([3, 3])

## $\langle \psi \vert H \vert \partial \psi \rangle$

This quantity is very common as the RHS of the variational quantum dynamics equation. And there is no good way to compute this quantity besides constructing corresponding Hadamard test circuit.

However, we can easily obtain this in AD framework, as long as ``stop_gradint`` API exists, which is the case for TensorCircuit. Namely, this quantity is obtained as $\partial (\langle \psi \vert H\vert \bot(\psi)\rangle)$, where the outside $\partial$ is automatically implemented by AD and $\bot$ is for ``stop_gradient`` op which stop the backpropagation.

In [19]:
z0 = tc.quantum.PauliStringSum2Dense([[3, 0, 0, 0, 0, 0]])


def h(thetas):
    w = psi(thetas)
    wr = K.stop_gradient(w)
    wl = K.conj(w)
    wl = K.reshape(wl, [1, -1])
    wr = K.reshape(wr, [-1, 1])
    e = wl @ z0 @ wr
    return K.real(e)[0, 0]

In [20]:
psi_h_partial_psi = K.grad(h)(thetas)
psi_h_partial_psi.shape

TensorShape([3])